# 연구 3단계: MNIST 데이터셋 환경에서 노이즈와 GMM 복잡도 관계 분석

## 연구 목표
실제 고차원 데이터인 MNIST 데이터셋의 잠재 공간(Latent Space)에서도, '노이즈 수준'과 '최적 컴포넌트 개수'($k_{opt}$) 사이의 관계가 통제된 환경에서와 같이 동일하게 나타나는지 실증적으로 검증한다.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from tensorflow.keras.datasets import mnist
from tqdm.notebook import tqdm
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)

## 1. 실험 조건 설정
모든 실험에서 동일하게 사용될 하이퍼파라미터를 정의합니다.

In [2]:
# 실험 반복 횟수 (통계적 신뢰도 확보)
N_RUNS = 30

# 테스트할 노이즈 수준 리스트 (이미지 픽셀 스케일 0-1 기준)
NOISE_LEVELS = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]

# 테스트할 GMM 컴포넌트 개수 범위
N_COMPONENTS_RANGE = range(1, 26)

# PCA로 축소할 차원 수
N_PCA_COMPONENTS = 32

## 2. MNIST 데이터 로딩 및 전처리
데이터를 로드하고, PCA 모델을 미리 학습시켜 둡니다.

In [3]:
# MNIST 데이터 로드
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_all = np.concatenate([x_train, x_test], axis=0)

# 0-1 스케일로 정규화 및 1D로 펼치기
x_all_flat = x_all.reshape(-1, 28*28) / 255.0

# 차원 축소를 위한 PCA 모델 학습
print(f"PCA 모델 학습 중... (차원: {N_PCA_COMPONENTS})")
pca = PCA(n_components=N_PCA_COMPONENTS, random_state=42)
pca.fit(x_all_flat)
print("PCA 모델 학습 완료.")

PCA 모델 학습 중... (차원: 32)
PCA 모델 학습 완료.


## 3. 핵심 연구 로직 실행
각 노이즈 수준에 대해 N_RUNS 만큼 실험을 반복하여, 최적의 컴포넌트 개수를 찾습니다.

In [4]:
results = []

for noise in tqdm(NOISE_LEVELS, desc="Noise Levels"):
    optimal_k_for_runs = []
    for run in tqdm(range(N_RUNS), desc=f"Runs for Noise {noise}", leave=False):
        # 1. 원본 이미지에 노이즈 주입
        # 매 실행마다 다른 노이즈 패턴을 위해 데이터 다시 로드
        x_all_flat_clean = x_all.reshape(-1, 28*28) / 255.0
        noise_matrix = np.random.randn(*x_all_flat_clean.shape) * noise
        x_all_flat_noisy = np.clip(x_all_flat_clean + noise_matrix, 0, 1)
        
        # 2. PCA를 이용해 차원 축소
        latent_vectors = pca.transform(x_all_flat_noisy)
        
        # 3. 각 k에 대해 GMM 학습 및 BIC 계산
        bics = []
        for k in N_COMPONENTS_RANGE:
            gmm = GaussianMixture(n_components=k, random_state=42, reg_covar=1e-6)
            gmm.fit(latent_vectors)
            bics.append(gmm.bic(latent_vectors))
            
        # 현재 실행(run)에서 최적의 k 찾기
        best_k = N_COMPONENTS_RANGE[np.argmin(bics)]
        optimal_k_for_runs.append(best_k)
        
    # 현재 노이즈 수준에 대한 통계치 계산
    mean_optimal_k = np.mean(optimal_k_for_runs)
    std_optimal_k = np.std(optimal_k_for_runs)
    
    results.append({
        'noise_level': noise,
        'mean_k_opt': mean_optimal_k,
        'std_k_opt': std_optimal_k,
        'all_k_opts': optimal_k_for_runs
    })

results_df_mnist = pd.DataFrame(results)

Noise Levels:   0%|          | 0/6 [00:00<?, ?it/s]

Runs for Noise 0.0:   0%|          | 0/30 [00:00<?, ?it/s]

KeyboardInterrupt: 

## 4. 결과 분석 및 시각화

In [ ]:
print("--- MNIST 실험 결과 ---")
print(results_df_mnist[['noise_level', 'mean_k_opt', 'std_k_opt']])

plt.figure(figsize=(12, 7))

plt.errorbar(
    results_df_mnist['noise_level'], 
    results_df_mnist['mean_k_opt'], 
    yerr=results_df_mnist['std_k_opt'],
    fmt='-D', # 마커 변경
    capsize=5,
    label='평균 최적 k (오차막대: 1-std)'
)

plt.title('노이즈 수준에 따른 GMM 최적 컴포넌트 개수 변화 (MNIST 잠재 공간)', fontsize=16)
plt.xlabel('노이즈 수준 (Noise Level)', fontsize=12)
plt.ylabel(f'최적 컴포넌트 개수 (평균, N={N_RUNS}회 실행)', fontsize=12)
plt.xticks(NOISE_LEVELS)
plt.grid(True, which='both', linestyle='--')
plt.legend()
plt.show()